### a) Chargement des données Mongo

In [ ]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient("mongodb://localhost:27017")
db = client.trackrapi

sessions = list(db.sessions.find({"stats": {"$exists": True}}))
df = pd.DataFrame([
    {
        "duration": s["stats"]["durationMs"] / 60000,
        "avgHr": s["stats"]["hrAvg"],
        "maxHr": s["stats"]["hrMax"],
        "distance": s["stats"]["distanceKm"],
        "steps": s["stats"]["steps"],
        "rmssd": s["stats"].get("medianRmssd", 30),
        "loadScore": s["stats"]["loadScoreBaseline"]
    }
    for s in sessions
])


### b) Comparaison modèles

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
X = df.drop("loadScore", axis=1)
y = df["loadScore"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf = RandomForestRegressor()
xgb = XGBRegressor(objective="reg:squarederror")

rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

print("RF MAE:", mean_absolute_error(y_test, rf.predict(X_test)))
print("XGB MAE:", mean_absolute_error(y_test, xgb.predict(X_test)))

### c) Sauvegarde du modèle

In [ ]:
import joblib
joblib.dump(xgb, "models/loadscore_xgb.pkl")
